# **Mixture of Agent**

# **Code ReRanking**

In [ ]:
!pip install -U langchain-cohere
!pip install cohere
!pip install deeplake
!pip install langchain
!pip install tiktoken
!pip install langchain_cohere
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.17
    Uninstalling langchain-community-0.2.17:
      Successfully uninstalled langchain-community-0.2.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependen

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

In [ ]:
from langchain.vectorstores.deeplake import DeepLake
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [ ]:
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key='Ef0G6b6IOqSKPbuSIUwUFAy3v2Is2K7g9ZQYjhn0',
)

docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    num_workers=4,
)

In [ ]:
file = "/content/codejava2.pdf"
loader = PyPDFLoader(file_path=file)
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
)
pages = loader.load()
chunks = text_splitter.split_documents(pages)
_ = docstore.add_documents(documents=chunks)

Creating 9 embeddings in 1 batches of size 9:: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


In [ ]:
docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    read_only=True,
    num_workers=4,
)

TEMPERATURE = 0.5
llm = ChatCohere(
    model="command",
    cohere_api_key='Ef0G6b6IOqSKPbuSIUwUFAy3v2Is2K7g9ZQYjhn0',
    temperature=TEMPERATURE,
)

Deep Lake Dataset in deeplake_vstore already exists, loading from the storage


In [ ]:
retriever = docstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "fetch_k": 20,
        "k": 10,
    }
)

In [ ]:
cohere_rerank = CohereRerank(
    cohere_api_key='Ef0G6b6IOqSKPbuSIUwUFAy3v2Is2K7g9ZQYjhn0',
    model="rerank-english-v3.0",
)
docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    read_only=True,
    num_workers=4,
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank,
    base_retriever=docstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "fetch_k": 20,
            "k": 15,
        },
    ),
)

Deep Lake Dataset in deeplake_vstore already exists, loading from the storage


In [ ]:
prompt_template = """You are an intelligent chatbot that can rearrange the code segment. You will be provided a code segment.
Your task is to rearrange the lines of code with respect to the purpose of the code and generate the rearranged code segment

Context: {context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT, "verbose": False}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
query = "rearrange the line of codes as per the purpose of the code."
response = qa.invoke({"query": query})
result = response["result"]
print(result)

I have carefully analyzed your code segment, and I have rearranged the lines of code to optimize its purpose. I have considered the step-by-step approach to execute the addition of a student in the database. My goal was to make it more readable and maintain a logical flow. 

Here is the rearranged code:
```java
package in.t4b.cucse.dao;

import java.sql.CallableStatement;
import java.sql.Connection;
import java.sql.ResultSet;
import java.util.ArrayList;

import in.t4b.cucse.db.DBCon;
import in.t4b.cucse.model.Course;
import in.t4b.cucse.model.Student;
import jakarta.servlet.http.HttpServletRequest;

public class StudentDao {

    public int addStudent(HttpServletRequest request) {
        int count = 0;
        Connection con = null;
        CallableStatement cs = null;

        try {
            con = DBCon.getConnection();
            String simpleProc = "{ call cucse_proc_student_create(?,?,?,?,?,?,?,?,?,?,?)}";
            cs = con.prepareCall(simpleProc);

            cs.setInt(1,

# **Sementic segmentation**

In [ ]:
!pip install -q groq
!pip install -q -U google-generativeai
! pip install langchain_community
! pip install --quiet langchain langchain_cohere langchain-openai tiktoken langchainhub langgraph
!pip install langchain
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/1

In [ ]:
# Optional: add tracing to visualize the agent trajectories
import os
from getpass import getpass

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_community.retrievers import CohereRagRetriever


In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
COHERE_API_KEY = userdata.get("COHERE_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")
GROQ_API_KEY = userdata.get("GROQ_API_KEY")

In [ ]:
os.environ['COHERE_API_KEY'] = COHERE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [ ]:
import operator
from typing import Annotated, Any, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import create_json_agent, initialize_agent, AgentType

from langchain.tools.tavily_search import TavilySearchResults
import json
from langchain_core.tools import tool

In [ ]:
search = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
tavily_tool = TavilySearchResults(api_wrapper=search)
command_R_llm=ChatCohere(
    model = "command-r-plus",
    cohere_api_key=COHERE_API_KEY, # Pass the API key directly
    MAX_TOKENS=32000)
# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    command_R_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


<ipython-input-6-cf0a48019305>:8: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent_chain = initialize_agent(


## **BINDING LLM WITH TOOLS. ALTERNATIVE TO THE SEARCH AGENT.**

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
print(os.environ.get('TAVILY_API_KEY'))
tavily_tool = TavilySearchResults(tavily_api_key=os.environ.get('TAVILY_API_KEY'))
tools = [tavily_tool]

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


In [ ]:

command_R_llm=ChatCohere(
    model = "command-r-plus",
    MAX_TOKENS=12000)
# initialize the agent
command_R_llm.bind_tools(tools)


RunnableBinding(bound=ChatCohere(client=<cohere.client.Client object at 0x7c536f4c39a0>, async_client=<cohere.client.AsyncClient object at 0x7c536f4c31f0>, model='command-r-plus', cohere_api_key=SecretStr('**********')), kwargs={'tools': [{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameter_definitions': {'query': {'description': 'search query to look up', 'type': 'str', 'required': True}}}]}, config={}, config_factories=[])

In [ ]:

class State(TypedDict):
    input: str
    responses: str
    feedback: str
    aggregated_response: Annotated[list, operator.add]
    final_answer: str
    iteration: int
    max_iterations: int
    reflection: str

# Initialize models and search tool
proposer_models = [
    ChatGroq(temperature=0, model_name="llama3-8b-8192",),
    #ChatGroq(temperature=0, model_name="meta-llama/Llama-2-7b-hf"),
    ChatGroq(temperature=0, model_name="gemma-7b-it"),
    ChatGroq(temperature=0, model_name="gemma2-9b-it"),
]
command_R_llm_ag=ChatCohere(
    model = "command-r-plus",
    MAX_TOKENS=32000)
aggregator_model = command_R_llm_ag
reflector_model = command_R_llm_ag



In [ ]:


class Proposer:
    def __init__(self, model: ChatOpenAI, id: str):
        self.model = model
        self.id = id

    def __call__(self, state: State) -> Dict[str, Dict[str, str]]:

        prompt = ChatPromptTemplate.from_messages([
            ("human", "Please provide a response to the following prompt: {input}\n\n"
             "The previous response was {response}\n\n"
             "The provided feedback was {feedback} \n\n"
                      "Always answer with as much detail as possible.")
        ])
        response = self.model(prompt.format_messages(
            input=state["input"],
            response=state['responses'],
            feedback=state['feedback']

        ))



        return {"aggregated_response": [json.dumps({self.id: response.content})]}




In [ ]:
def aggregator(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        #("human", "Given the following responses to the prompt '{input}', please synthesize them into a single, high-quality response:\n\n{responses}")
        ("human", '''
        You have been provided with a set of responses from various open-source models to the  user query '{input}'.
        Your task is to synthesize these responses into a single, high-quality response.
        It is crucial to critically evaluate the information provided in these responses,
        recognizing that some of it may be biased or incorrect.
        Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

        Responses from models:
        \n\n{responses}

''')
    ])
    response = aggregator_model(prompt.format_messages(
        input=state["input"],
        responses="\n\n".join(state["aggregated_response"])
    ))
    state["responses"] = response.content
    return state

In [ ]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser


class GradeGeneration(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    score: str = Field(
        description="Is this the correct answer to the question, 'yes' or 'no'"
    )
    feedback: str = Field(
        description="Provided specific feedback for improvement"
    )

def reflector(state: State) -> State:
    parser = PydanticOutputParser(pydantic_object=GradeGeneration)

    prompt  = ChatPromptTemplate.from_messages([
            ("human", "Given the following answer to the question: '{input}'\n\n"
            "Answer: {aggregated_response}\n\n"
                      "If the answer is satisfactory and complete, grade it as yes. \n"
    "Provide json object with  feedback string and binary score 'yes' or 'no' score to indicate whether the answer is correct"
            )
        ])
    chain = prompt | reflector_model | parser

    response = chain.invoke({
        'input':state["input"],
        'aggregated_response':state["responses"]
    })
    state["reflection"] = response.score
    state['feedback'] = response.feedback
    state["iteration"] += 1
    return state

In [ ]:
def should_continue(state: State) -> str:
    if state["iteration"] >= state["max_iterations"]:# or "YES" in state["reflection"].upper():
        state["final_answer"] = state["responses"]
        return "end"
    return "refine"

# Create the graph
workflow = StateGraph(State)

# Add nodes
for i, model in enumerate(proposer_models):
    workflow.add_node(f"proposer_{i}", Proposer(model, f"proposer_{i}"))
workflow.add_node("aggregator", aggregator)
workflow.add_node("reflector", reflector)

# Define edges
workflow.set_entry_point("proposer_0")
for i in range(1, len(proposer_models)):
    workflow.add_edge(f"proposer_0", f"proposer_{i}")
    workflow.add_edge(f"proposer_{i}", "aggregator")
workflow.add_edge("aggregator", "reflector")
workflow.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "refine": "proposer_0",
        "end": END
    }
)

In [ ]:
app = workflow.compile()

In [ ]:
codes = result

In [ ]:
codes = """
import java.util.Scanner;
public class MathOperations {
public int add(int a, int b) {
 return a + b;
}

public int multiply(int a, int b) {
return a * b;
}

public int add(int a, int b, int c) {
return a + b + c;
}
public static void main(String[] args) {
Scanner scanner = new Scanner(System.in);
MathOperations math = new MathOperations();
System.out.print("Enter the first number: ");
int num1 = scanner.nextInt();
System.out.print("Enter the second number: ");
int num2 = scanner.nextInt();
System.out.print("Enter the third number: ");
int num3 = scanner.nextInt();
int sum1 = math.add(num1, num2);
System.out.println("Sum of " + num1 + " and " + num2 + ": " + sum1);
int sum2 = math.add(num1, num2, num3);
System.out.println("Sum of " + num1 + ", " + num2 + ", and " + num3 + ": " + sum2);
int product = math.multiply(num1, num2);
System.out.println("Product of " + num1 + " and " + num2 + ": " + product);
scanner.close();
}
}

generate a report of the code containing code analysis, complexity analyis ,commented code,sample output,extra remarks on code


"""

In [ ]:
def query_moa(question):
    # Run the graph
    initial_state = {
        "input": question,
        "responses": '',
        'feedback':"",
        "aggregated_response": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3,
        "reflection": ""
    }
    for output in app.stream(initial_state):
        for key, value in output.items():
            print(f"Finished running: {key}:")
    print(value['responses'])
    return value

In [ ]:
message = "You are an advanced AI assistant specialized in creating highly effective semantic segmentation. You are capable of segmenting a given code snippet based on the following conditions: 1. Identify and group methods that have similar functionality into clusters. This should be done by analyzing the code and identifying common functionality, such as input parameters, output parameters, and method calls. 2. Separate different functionality into separate clusters. This can be done by identifying code blocks that are not related to any of the existing clusters and grouping them together. 3. Cluster header files together based on their dependencies. This can be done by analyzing the header files and identifying which files are required by other files. 4. Isolate the main method for individual examination. This can be done by identifying the main method and separating it from the rest of the code. The tool should be capable of handling code snippets of varying sizes and complexity, and should provide accurate and reliable results. It should also be able to handle different programming languages and platforms. The ultimate goal of this tool is to provide a more efficient and effective way of analyzing code, which can help developers identify potential issues and improve the overall quality of their code."+codes

value = query_moa(message)

<ipython-input-10-57d8be89f88b>:14: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = self.model(prompt.format_messages(


Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
## Comprehensive Code Analysis Report: MathOperations.java

### Overview:
This report offers a comprehensive analysis of the provided Java code snippet, emphasizing semantic segmentation, complexity, and code quality. The code defines a class, 'MathOperations', which facilitates basic arithmetic operations, encompassing addition and multiplication.

### Semantic Segmentation:
- **Method Clustering:** The code incorporates three methods associated with addition: 'add(int, int)' and 'add(int, int, int)'. These methods are grouped together based on t

# Report generation

In [ ]:
search = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
tavily_tool = TavilySearchResults(api_wrapper=search)
command_R_llm=ChatCohere(
    model = "command-r-plus",
    cohere_api_key=COHERE_API_KEY, # Pass the API key directly
    MAX_TOKENS=32000)
# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    command_R_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
print(os.environ.get('TAVILY_API_KEY'))
tavily_tool = TavilySearchResults(tavily_api_key=os.environ.get('TAVILY_API_KEY'))
tools = [tavily_tool]

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


In [ ]:

command_R_llm=ChatCohere(
    model = "command-r-plus",
    MAX_TOKENS=12000)
# initialize the agent
command_R_llm.bind_tools(tools)


RunnableBinding(bound=ChatCohere(client=<cohere.client.Client object at 0x7c536f212380>, async_client=<cohere.client.AsyncClient object at 0x7c536f213be0>, model='command-r-plus', cohere_api_key=SecretStr('**********')), kwargs={'tools': [{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameter_definitions': {'query': {'description': 'search query to look up', 'type': 'str', 'required': True}}}]}, config={}, config_factories=[])

In [ ]:

class State(TypedDict):
    input: str
    responses: str
    feedback: str
    aggregated_response: Annotated[list, operator.add]
    final_answer: str
    iteration: int
    max_iterations: int
    reflection: str

# Initialize models and search tool
proposer_models = [
    ChatGroq(temperature=0, model_name="llama3-8b-8192",),
    #ChatGroq(temperature=0, model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"),
    ChatGroq(temperature=0, model_name="gemma-7b-it"),
    ChatGroq(temperature=0, model_name="gemma2-9b-it"),
]
command_R_llm_ag=ChatCohere(
    model = "command-r-plus",
    MAX_TOKENS=32000)
aggregator_model = command_R_llm_ag
reflector_model = command_R_llm_ag



In [ ]:


class Proposer:
    def __init__(self, model: ChatOpenAI, id: str):
        self.model = model
        self.id = id

    def __call__(self, state: State) -> Dict[str, Dict[str, str]]:

        prompt = ChatPromptTemplate.from_messages([
            ("human", "Please provide a response to the following prompt: {input}\n\n"
             "The previous response was {response}\n\n"
             "The provided feedback was {feedback} \n\n"
                      "Always answer with as much detail as possible.")
        ])
        response = self.model(prompt.format_messages(
            input=state["input"],
            response=state['responses'],
            feedback=state['feedback']

        ))



        return {"aggregated_response": [json.dumps({self.id: response.content})]}




In [ ]:
def aggregator(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        #("human", "Given the following responses to the prompt '{input}', please synthesize them into a single, high-quality response:\n\n{responses}")
        ("human", '''
        You have been provided with a set of responses from various open-source models to the  user query '{input}'.
        Your task is to synthesize these responses into a single, high-quality response.
        It is crucial to critically evaluate the information provided in these responses,
        recognizing that some of it may be biased or incorrect.
        Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

        Responses from models:
        \n\n{responses}

''')
    ])
    response = aggregator_model(prompt.format_messages(
        input=state["input"],
        responses="\n\n".join(state["aggregated_response"])
    ))
    state["responses"] = response.content
    return state

In [ ]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser


class GradeGeneration(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    score: str = Field(
        description="Is this the correct answer to the question, 'yes' or 'no'"
    )
    feedback: str = Field(
        description="Provided specific feedback for improvement"
    )

def reflector(state: State) -> State:
    parser = PydanticOutputParser(pydantic_object=GradeGeneration)

    prompt  = ChatPromptTemplate.from_messages([
            ("human", "Given the following answer to the question: '{input}'\n\n"
            "Answer: {aggregated_response}\n\n"
                      "If the answer is satisfactory and complete, grade it as yes. \n"
    "Provide json object with  feedback string and binary score 'yes' or 'no' score to indicate whether the answer is correct"
            )
        ])
    chain = prompt | reflector_model | parser

    response = chain.invoke({
        'input':state["input"],
        'aggregated_response':state["responses"]
    })
    state["reflection"] = response.score
    state['feedback'] = response.feedback
    state["iteration"] += 1
    return state

In [ ]:
def should_continue(state: State) -> str:
    if state["iteration"] >= state["max_iterations"]:# or "YES" in state["reflection"].upper():
        state["final_answer"] = state["responses"]
        return "end"
    return "refine"

# Create the graph
workflow = StateGraph(State)

# Add nodes
for i, model in enumerate(proposer_models):
    workflow.add_node(f"proposer_{i}", Proposer(model, f"proposer_{i}"))
workflow.add_node("aggregator", aggregator)
workflow.add_node("reflector", reflector)

# Define edges
workflow.set_entry_point("proposer_0")
for i in range(1, len(proposer_models)):
    workflow.add_edge(f"proposer_0", f"proposer_{i}")
    workflow.add_edge(f"proposer_{i}", "aggregator")
workflow.add_edge("aggregator", "reflector")
workflow.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "refine": "proposer_0",
        "end": END
    }
)

In [ ]:
app = workflow.compile()

In [ ]:
def query_moa(question):
    # Run the graph
    initial_state = {
        "input": question,
        "responses": '',
        'feedback':"",
        "aggregated_response": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3,
        "reflection": ""
    }
    for output in app.stream(initial_state):
        for key, val in output.items():
            print(f"Finished running: {key}:")
    print(val['responses'])
    return val

In [ ]:
print(value['responses'])

## Comprehensive Code Analysis Report: MathOperations.java

### Overview:
This report offers a comprehensive analysis of the provided Java code snippet, emphasizing semantic segmentation, complexity, and code quality. The code defines a class, 'MathOperations', which facilitates basic arithmetic operations, encompassing addition and multiplication.

### Semantic Segmentation:
- **Method Clustering:** The code incorporates three methods associated with addition: 'add(int, int)' and 'add(int, int, int)'. These methods are grouped together based on their shared functionality and similar input/output parameters. The 'multiply(int, int)' method constitutes a distinct cluster, indicating a separate functionality.
- **Non-Clustered Code:** The code block responsible for user interaction, including input collection and result display, is not assigned to any specific cluster. It acts as an intermediary between the clustered methods and user interface.
- **Header File Clustering:** Given the abs

In [ ]:
message = "You are an advanced AI assistant specialized in creating highly effective report generation of a code segment. You are capable of generating proper report of  a given code snippet based on semantic segmentation. Report should include 1. Well commented code . 2. Sample output for at least 2 different types of input. 3. Prerequisite for the code implementation. 4. Specific data structure used if any 5.Complexity Analysis of the code . It should also be able to handle different programming languages and platforms. The ultimate goal of this tool is to provide a more efficient and effective way of analyzing code, which can help user to understand the code properly along with it's specific purpose."+ value['responses'] +codes

val = query_moa(message)

Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_1:
Finished running: proposer_2:
Finished running: aggregator:
Finished running: reflector:
Finished running: proposer_0:
Finished running: proposer_2:
Finished running: proposer_1:
Finished running: aggregator:
Finished running: reflector:
## Comprehensive Code Analysis Report: MathOperations.java

### Overview:
This report offers a thorough analysis of the provided Java code snippet, emphasizing semantic segmentation, complexity, and code quality. The code defines a class, 'MathOperations', designed to execute basic arithmetic operations, specifically addition and multiplication.

### Semantic Segmentation:
- **Method Clustering:** The code exhibits a clear clustering of methods based on functionality. There are two distinct clusters: the addition cluster, comprising 'add(int, int)

In [ ]:
print(val['responses'])

## Comprehensive Code Analysis Report: MathOperations.java

### 1. Overview
This report provides a comprehensive analysis of the Java code snippet "MathOperations.java," focusing on its structure, functionality, complexity, and potential areas for improvement. The code defines a class named `MathOperations` that encapsulates basic mathematical operations, including addition and multiplication. 

### 2. Code Functionality
The `MathOperations` class offers three primary methods:
- **`add(int a, int b)`:** This method calculates the sum of two integers (`a` and `b`) and returns the result.
- **`multiply(int a, int b)`:** This method calculates the product of two integers (`a` and `b`) and returns the result.
- **`add(int a, int b, int c)`:** This is an overloaded version of the `add` method, which calculates the sum of three integers (`a`, `b`, and `c`) and returns the result.

The `main` method serves as the entry point for the program. It prompts the user to enter three numbers, creates

# Memory Usage

In [ ]:
!pip install mem0ai
!pip install pyautogen mem0ai openai
!pip -q install --upgrade together
!pip install --upgrade langchain-together
!pip install --upgrade langchain
!pip install --upgrade openai
!pip install cloudpickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully

In [ ]:
%pip install -qU langchain-groq

In [ ]:
from mem0 import MemoryClient
client = MemoryClient(api_key="m0-iJYAA4DB09dHhP6rVqMI3EGPRWLabcMyslnIHGWy")

In [ ]:
client.delete_all(user_id="output")

{'message': 'Memories deleted successfully!'}

In [ ]:
from os import environ
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_together import ChatTogether
from autogen import ConversableAgent
from mem0 import MemoryClient
from langchain_groq import ChatGroq
load_dotenv()

os.environ["GROQ_API_KEY"] ="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"
#os.environ["OPENAI_API_KEY"]="sk-iKVYep8Co__PXo1z0X9pmE3Ja6tu_1zOqicFqwka7zT3BlbkFJSvZUw2Tf_RxxGl6fY3xCDLhRyd69mmueLnDRox_oIA"

template = """
You are a helpful assistant. Help the user with their requests.
"""

prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{query}"),
        ]
    )

memory = ConversationBufferMemory(memory_key="history", return_messages=True)

llm_chain = LLMChain(
    llm=ChatGroq(model="mixtral-8x7b-32768"),
    prompt=prompt_template,
    verbose=False,
    memory=memory,
)

query_count=1
print("Do you want to give feedback ?(1 for feedback or 0 for exit)")
t=int(input())
while(t):
    if(t==1):
        print("Enter your feedback:")
        feedback=input()
        response = llm_chain.run(feedback)
        print(feedback)
        print(response)
        messages = [
            {"role": "user", "content": feedback},
            {"role": "assistant", "content": response}
        ]
        if(feedback.find("output")>0):client.add(messages, user_id="output", output_format="v1.1")
        elif(feedback.find("complexity")>0):client.add(messages, user_id="complexity", output_format="v1.1")
    elif(t==2):
        print("Enter your query:")
        query=input()
        q1=client.search(query, user_id="output", output_format="v1.1")
        print(q1)
        q2=client.search(query, user_id="complexity", output_format="v1.1")
        print(q2)
        x1=q1['results']
        x2=q2['results']
        if(len(x2)==0):
          input_string=query+" previous report:"+val['responses']+"feedback :"+x1[0]['memory']
        elif(len(x1)==0):
          input_string=query+" previous report: "+val['responses']+"feedback :"+x2[0]['memory']
        else:
          input_string=query+" previous report: "+val['responses']+"feedback :"+x1[0]['memory']+x2[0]['memory']
        response = llm_chain.run(input_string)
        #print(val['responses']+x1[0]['memory']+x2[0]['memory'])
        print(response)
        break

    print("Do you want to give feedback or want modified report?(1 for feedback or 2 for report)")
    t=int(input())

print("exit")


Do you want to give feedback ?(1 for feedback or 0 for exit)
1
Enter your feedback:
Hi! Whenever I ask for sample output always give the output for 3 different set inputs covering all range of inputs
Hi! Whenever I ask for sample output always give the output for 3 different set inputs covering all range of inputs
Hello! I'd be happy to help you with that. Going forward, I will provide three different sample outputs for a given function or code snippet, using a range of inputs to demonstrate its behavior. Please let me know if you have any specific requests or examples in mind.

For instance, if you ask for the sample output of a function that calculates the area of a circle, I would provide outputs for radii of 1, 5, and 50 to show how the function handles small, medium, and large input values. This way, you can better understand how the function or code snippet behaves in various scenarios.
Do you want to give feedback or want modified report?(1 for feedback or 2 for report)
2
Enter 

# Report PDF generation

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=55bec771e45764ef25b0f5e2db0a08a6a4d96bb128e15a5bbd917967045fabac
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import fpdf

# Assuming x["output"] contains the agent's response as a string
response_text = str(response)
#print(response_text)
# Create a new PDF object
pdf = fpdf.FPDF()

# Add a page
pdf.add_page()

# Set the font and font size
pdf.set_font("Arial", size=12)

# Split the response into lines
lines = response_text.split("\n")

# Loop through each line and add it to the PDF
pdf.multi_cell(0, 10, txt=response_text)

# Save the PDF file
pdf.output("code_report.pdf")

''